# Telco Cusstomer Churn: Feature Engineering

In [106]:
# Load neccesory libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [107]:
# Load dataset
df = pd.read_csv(r"C:\Users\linto\Code\churn-x\telcoData\cleaned_Telco_customer_churn.csv")

In [108]:
# checking first 3 rows of dataset
df.head(3)

,Country,State,City,Zip Code,Latitude,Longitude,Gender,Senior Citizen,Partner,Dependents,...,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Total Charges,Churn Label,Churn Value,Churn Score,CLTV
0,United States,California,Los Angeles,90003,33.964131,-118.272783,Male,No,No,No,...,No,Month-to-month,Yes,Mailed check,53.85,108,Yes,1,86,3239
1,United States,California,Los Angeles,90005,34.059281,-118.307420,Female,No,No,Yes,...,No,Month-to-month,Yes,Electronic check,70.70,151,Yes,1,67,2701
2,United States,California,Los Angeles,90006,34.048013,-118.293953,Female,No,No,Yes,...,Yes,Month-to-month,Yes,Electronic check,99.65,820,Yes,1,86,5372


In [109]:
# find the unique values in each col
for col in df.columns:
    if df[col].nunique() <= 5:
        print(f"{col} -> {df[col].nunique()} -> {df[col].unique()}")
    else:
        print(f"{col} -> {df[col].nunique()}")

Country -> 1 -> ['United States']
State -> 1 -> ['California']
City -> 1129
Zip Code -> 1652
Latitude -> 1652
Longitude -> 1651
Gender -> 2 -> ['Male' 'Female']
Senior Citizen -> 2 -> ['No' 'Yes']
Partner -> 2 -> ['No' 'Yes']
Dependents -> 2 -> ['No' 'Yes']
Tenure Months -> 73
Phone Service -> 2 -> ['Yes' 'No']
Multiple Lines -> 3 -> ['No' 'Yes' 'No phone service']
Internet Service -> 3 -> ['DSL' 'Fiber optic' 'No']
Online Security -> 3 -> ['Yes' 'No' 'No internet service']
Online Backup -> 3 -> ['Yes' 'No' 'No internet service']
Device Protection -> 3 -> ['No' 'Yes' 'No internet service']
Tech Support -> 3 -> ['No' 'Yes' 'No internet service']
Streaming TV -> 3 -> ['No' 'Yes' 'No internet service']
Streaming Movies -> 3 -> ['No' 'Yes' 'No internet service']
Contract -> 3 -> ['Month-to-month' 'Two year' 'One year']
Paperless Billing -> 2 -> ['Yes' 'No']
Payment Method -> 4 -> ['Mailed check' 'Electronic check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Monthly Charges -> 1

- ```Country``` and ```State``` have only one unique value, so they can be removed
- ```Churn Value``` and ```Churn Score``` represent the same information, so one can be dropped
- Since we already have ```City``` and ```Zip Code```, we can remove ```Latitude``` and ```Longitude```
- Removing ```Total Charges```. It is ```Tenure Months``` * ```Monthly Charges```
- Removing ```Zip Code``` and ```Gender``` to remove gender bias and location bias

In [110]:
df2 = df.drop(['Country','State','Churn Label','Latitude','Longitude', 'Total Charges', 'Zip Code', 'Gender'], axis=1)

In [111]:
df2.head(3)

,City,Senior Citizen,Partner,Dependents,Tenure Months,Phone Service,Multiple Lines,Internet Service,Online Security,Online Backup,...,Tech Support,Streaming TV,Streaming Movies,Contract,Paperless Billing,Payment Method,Monthly Charges,Churn Value,Churn Score,CLTV
0,Los Angeles,No,No,No,2,Yes,No,DSL,Yes,Yes,...,No,No,No,Month-to-month,Yes,Mailed check,53.85,1,86,3239
1,Los Angeles,No,No,Yes,2,Yes,No,Fiber optic,No,No,...,No,No,No,Month-to-month,Yes,Electronic check,70.70,1,67,2701
2,Los Angeles,No,No,Yes,8,Yes,Yes,Fiber optic,No,No,...,No,Yes,Yes,Month-to-month,Yes,Electronic check,99.65,1,86,5372


In [112]:
import pandas as pd

# Example: df has 'city' and 'churn' columns
# churn should be 0/1

# Step 1: Compute mean churn per city
city_mean_churn = df2.groupby('City')['Churn Value'].mean()

# Step 2: Map this back to the dataframe
df2['city_encoded'] = df2['City'].map(city_mean_churn)

# ----> save the mapping in an encoder <------

# Step 3: Check correlation with churn
correlation = df2['city_encoded'].corr(df2['Churn Value'])
print("Correlation between city (encoded) and churn:", correlation)




Correlation between city (encoded) and churn: 0.4185068451014711


## 1. Feature Encoding

### 1.1 Encode Text Columns

In [113]:
nominal_cols =['Senior Citizen', 'Partner', 'Dependents', 'Phone Service', 'Multiple Lines', 
               'Internet Service', 'Online Security', 'Online Backup', 'Device Protection', 'Tech Support', 'Streaming TV',
               'Streaming Movies', 'Contract', 'Paperless Billing', 'Payment Method']

# 'City', 

In [114]:
df2.drop('City', axis=1, inplace=True)
df3 = pd.get_dummies(df2, columns=nominal_cols, drop_first=True, dtype=int)

In [115]:
df3.head(3)

,Tenure Months,Monthly Charges,Churn Value,Churn Score,CLTV,city_encoded,Senior Citizen_Yes,Partner_Yes,Dependents_Yes,Phone Service_Yes,...,Streaming TV_No internet service,Streaming TV_Yes,Streaming Movies_No internet service,Streaming Movies_Yes,Contract_One year,Contract_Two year,Paperless Billing_Yes,Payment Method_Credit card (automatic),Payment Method_Electronic check,Payment Method_Mailed check
0,2,53.85,1,86,3239,0.295082,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
1,2,70.70,1,67,2701,0.295082,0,0,1,1,...,0,0,0,0,0,0,1,0,1,0
2,8,99.65,1,86,5372,0.295082,0,0,1,1,...,0,1,0,1,0,0,1,0,1,0


## 2. Feature Selection

### 2.1 Correlation

### 2.2 VIF

In [116]:
from statsmodels.stats.outliers_influence import variance_inflation_factor

def calculate_vif(data):
    vif_df = pd.DataFrame()
    vif_df['Column'] = data.columns
    vif_df['VIF'] = [variance_inflation_factor(data.values,i) for i in range(data.shape[1])]
    return vif_df

In [117]:
df_vif = calculate_vif(df3)
df_vif.sort_values(by='VIF', ascending=False)

,Column,VIF
13,Internet Service_No,inf
14,Online Security_No internet service,inf
18,Device Protection_No internet service,inf
24,Streaming Movies_No internet service,inf
20,Tech Support_No internet service,inf
22,Streaming TV_No internet service,inf
16,Online Backup_No internet service,inf
9,Phone Service_Yes,1781.697860
1,Monthly Charges,863.399909
12,Internet Service_Fiber optic,148.451378


In [118]:
def reduce_vif(df, threshold=10.0):
    """
    Iteratively removes features with VIF above threshold.
    
    Parameters:
        df (DataFrame): Input dataframe with only numeric features.
        threshold (float): Maximum allowed VIF.
    
    Returns:
        DataFrame: Reduced dataframe with all VIF <= threshold
        DataFrame: Final VIF table
    """
    df_clean = df.copy()
    
    while True:
        vif_df = calculate_vif(df_clean)
        vif_df = vif_df.sort_values(by="VIF", ascending=False).reset_index(drop=True)
        
        # check highest VIF
        max_vif = vif_df.loc[0, "VIF"]
        if max_vif > threshold:
            drop_col = vif_df.loc[0, "Column"]
            print(f"Dropping '{drop_col}' with VIF={max_vif:.2f}")
            df_clean = df_clean.drop(columns=[drop_col])
        else:
            break
    
    return df_clean, vif_df


In [119]:
import warnings

# Ignore specific runtime warnings (like divide by zero in VIF)
warnings.filterwarnings("ignore", category=RuntimeWarning)

# Now run your VIF function
df4, vif_df = reduce_vif(df3.drop('Churn Value', axis=1), threshold=10.0)

Dropping 'Online Security_No internet service' with VIF=inf
Dropping 'Internet Service_No' with VIF=inf
Dropping 'Streaming Movies_No internet service' with VIF=inf
Dropping 'Tech Support_No internet service' with VIF=inf
Dropping 'Streaming TV_No internet service' with VIF=inf
Dropping 'Device Protection_No internet service' with VIF=inf
Dropping 'Phone Service_Yes' with VIF=1781.23
Dropping 'Monthly Charges' with VIF=93.80
Dropping 'CLTV' with VIF=12.14


In [120]:
vif_df

,Column,VIF
0,Tenure Months,7.460829
1,Churn Score,7.038049
2,Internet Service_Fiber optic,3.441484
3,Contract_Two year,3.432528
4,city_encoded,3.301218
5,Online Backup_No internet service,2.869935
6,Paperless Billing_Yes,2.824613
7,Streaming Movies_Yes,2.672181
8,Streaming TV_Yes,2.641071
9,Partner_Yes,2.584543


In [121]:
df4.head(3)

,Tenure Months,Churn Score,city_encoded,Senior Citizen_Yes,Partner_Yes,Dependents_Yes,Multiple Lines_No phone service,Multiple Lines_Yes,Internet Service_Fiber optic,Online Security_Yes,...,Device Protection_Yes,Tech Support_Yes,Streaming TV_Yes,Streaming Movies_Yes,Contract_One year,Contract_Two year,Paperless Billing_Yes,Payment Method_Credit card (automatic),Payment Method_Electronic check,Payment Method_Mailed check
0,2,86,0.295082,0,0,0,0,0,0,1,...,0,0,0,0,0,0,1,0,0,1
1,2,67,0.295082,0,0,1,0,0,1,0,...,0,0,0,0,0,0,1,0,1,0
2,8,86,0.295082,0,0,1,0,1,1,0,...,1,0,1,1,0,0,1,0,1,0


In [122]:
# cm = df4.corr()

# plt.figure(figsize=(20,12))
# sns.heatmap(cm, annot=True)
# plt.xticks(rotation=45, ha='right')
# plt.yticks(rotation=0)
# plt.tight_layout()
# plt.show()

In [123]:
# Compute correlations with Churn Score
corr_with_churn = df4.corr()['Churn Score'].sort_values(ascending=False)

# Convert to DataFrame for better display
corr_table = corr_with_churn.reset_index()
corr_table.columns = ['Feature', 'Correlation with Churn Score']

print(corr_table)


                                   Feature  Correlation with Churn Score
0                              Churn Score                      1.000000
1                             city_encoded                      0.288310
2             Internet Service_Fiber optic                      0.208687
3          Payment Method_Electronic check                      0.194603
4                    Paperless Billing_Yes                      0.129334
5                       Senior Citizen_Yes                      0.102181
6                     Streaming Movies_Yes                      0.048252
7                       Multiple Lines_Yes                      0.043252
8                         Streaming TV_Yes                      0.040869
9          Multiple Lines_No phone service                     -0.024136
10                       Online Backup_Yes                     -0.046879
11                   Device Protection_Yes                     -0.054870
12             Payment Method_Mailed check         

In [124]:
X = df4.drop('Churn Score', axis='columns')
y = df4['Churn Score']

from sklearn.preprocessing import MinMaxScaler

cols_to_scale = ['Tenure Months']
scaler = MinMaxScaler()

X[cols_to_scale] = scaler.fit_transform(X[cols_to_scale])
X.describe()

,Tenure Months,city_encoded,Senior Citizen_Yes,Partner_Yes,Dependents_Yes,Multiple Lines_No phone service,Multiple Lines_Yes,Internet Service_Fiber optic,Online Security_Yes,Online Backup_No internet service,...,Device Protection_Yes,Tech Support_Yes,Streaming TV_Yes,Streaming Movies_Yes,Contract_One year,Contract_Two year,Paperless Billing_Yes,Payment Method_Credit card (automatic),Payment Method_Electronic check,Payment Method_Mailed check
count,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,...,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.449599,0.265370,0.162147,0.483033,0.231010,0.096834,0.421837,0.439585,0.286668,0.216669,...,0.343888,0.290217,0.384353,0.387903,0.209144,0.240664,0.592219,0.216101,0.335794,0.228880
std,0.341104,0.184796,0.368612,0.499748,0.421508,0.295752,0.493888,0.496372,0.452237,0.412004,...,0.475038,0.453895,0.486477,0.487307,0.406726,0.427517,0.491457,0.411613,0.472301,0.420141
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.125000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.402778,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000
75%,0.763889,0.333333,0.000000,1.000000,0.000000,0.000000,1.000000,1.000000,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
